## Summary assignment - part 1

### Odeya Hazani- 207288457
### Roni Epstein- 211645825

In [1]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

In [2]:
# Function to calculate the number of days until the start of a given month and year
def days_until_month_start(year, month):
    today = datetime.today()
    first_day_of_month = datetime(year, month, 1)
    difference = first_day_of_month - today
    return difference.days

In [3]:
# Define the URL of the website and the car model to search for
car_model ="שברולט"
url = "https://www.ad.co.il/car"

In [4]:
# Send a request to the website and parse the content
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

In [ ]:
# Find the section containing the car model
car_shev = soup.find_all("div", class_="fast-filters-children d-flex flex-row align-items-center me-3 mb-1")

In [5]:
# Extract the URL for the specific car model
for element in car_shev:
    anchor_tags = element.find_all("a")  # Find all anchor tags within this element
    for tag in anchor_tags:
        if car_model in tag.text:
            tag_model = tag['href']
            keywords_model = tag_model[tag_model.find("?"):]
            
keywords_model

'?sp261=13891'

__________________________

In [ ]:
# Send a request to the car model's URL and parse the content
response = requests.get(url+keywords_model)
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
# Extract pagination URLs
pagination = soup.find('ul', class_="pagination justify-content-between justify-content-sm-center flex-wrap")
page_items = pagination.find_all('li', class_="page-item")
page_urls_all = [item.find('a')['href'] for item in page_items if item.find('a')]
page_urls = list(set(page_urls_all))
page_urls

['/car?sp261=13891&pageindex=2',
 '/car?sp261=13891&pageindex=4',
 '/car?sp261=13891',
 '/car?sp261=13891&pageindex=3']

In [7]:
# Function to scrape data IDs from each page
def scrape_data_ids(url_model):
    url = "https://www.ad.co.il"
    response = requests.get(url_model)
    soup = BeautifulSoup(response.text, 'html.parser')
    cards_container = soup.find('div', class_='cards-wrap s m l')
    data_ids = []
    cards = cards_container.find_all('div', class_='card-block')
    for card in cards:
        data_id = card.get('data-id')
        if data_id:  # Check if data-id is not None
            data_id = url + "/ad/" + str(data_id)
            data_ids.append(data_id)
    return data_ids

data_ids_pags = []

Total data IDs collected: 183


In [ ]:
# Collect data IDs from all pages
for page_url in page_urls:
    full_url = 'https://www.ad.co.il' + page_url
    data_ids = scrape_data_ids(full_url)
    data_ids_pags.extend(data_ids)

print(f"Total data IDs collected: {len(data_ids_pags)}")

In [8]:
# Function to scrape car data from each car's page
def scrape_car_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    manufactor = car_model
    cars_card_div = soup.find('div', class_="d-flex justify-content-between") 
    cars_card = cars_card_div.find_all('h2', class_="card-title")
    if len(cars_card) == 1:
        price = None
    else:
        price = float(cars_card[1].get_text()[:-1].replace(',', '').replace('₪', '').strip()) if cars_card else None
    
    model_string = cars_card[0].get_text().split() if cars_card else 'N/A'  
    model = ' '.join(model_string[1:])

    cars_table = soup.find('table', class_="table table-sm mb-4")
    rows = cars_table.find_all('tr') if cars_table else []
       
    # Initialize variables 
    year = None
    hand = None
    gear = 'N/A'
    engine_capacity = None
    engine_type = 'N/A'
    area = 'N/A'
    city = 'N/A'     
    prev_ownership =  'N/A'
    curr_ownership = 'N/A'
    color = 'N/A'
    test_date = None
    km = None
        
    # Extract data from the table
    for row in rows:
        cells = row.find_all('td')
        if len(cells) == 2:
            key = cells[0].text.strip()
            value = cells[1].text.strip()

            if key == 'שנה':
                year = int(value)
            elif key == 'יד':
                hand = int(value)   
            elif key == 'ת. הילוכים':
                gear = value
            elif key == 'נפח':
                engine_capacity = int(value.replace(',', ''))  # Remove comma from engine capacity
            elif key == 'סוג מנוע':
                engine_type = value
            elif key == 'אזור':
                area = value
            elif key == 'עיר':
                city = value
            elif key == 'ק"מ':
                km = int(value.replace(',', ''))
            elif key == 'צבע':
                color = value
            elif key == 'בעלות קודמת':
                prev_ownership = value
            elif key == 'בעלות נוכחית':
                curr_ownership = value
            elif key == 'טסט עד':
                test_date = value
    
    test = days_until_month_start(int(test_date.split("/")[1]),int(test_date.split("/")[0])) if test_date else None
    cre_date =soup.find_all('div',class_="px-3")[0].get_text().split()[-1] if soup.find_all('div',class_="px-3")[0] else 'N/A'
    repub_date = soup.find_all('div',class_="px-3")[1].get_text().split()[-1] if soup.find_all('div',class_="px-3")[0] else 'N/A'
    description = soup.find('p',class_="text-word-break").get_text().replace('\n', '').replace('\r', ' ') if soup.find('p',class_="text-word-break") else 'N/A'
    pic_num = len(soup.find_all('div', class_='justify-content-center px-1')) if soup.find_all('div', class_='justify-content-center px-1') else None
    supply_score = None

    # Add each car's data as a list to the car_data list
    car_data =[manufactor, year, model, hand, gear, engine_capacity, engine_type, prev_ownership,
                      curr_ownership, area, city, price, pic_num, cre_date, repub_date, description, color, km,test, supply_score]
    return car_data

In [9]:
# Create an empty DataFrame with the specified columns
df = pd.DataFrame(columns=['manufactor','Year','model','Hand','Gear','capacity_Engine','Engine_type','Prev_ownership',
                                         'Curr_ownership','Area','City','Price','Pic_num','Cre_date','Repub_date','Description','Color','Km',
                                         'Test','Supply_score'])

In [10]:
# Scrape car data for each car and add to the DataFrame
list_cars = []
for car_url in data_ids_pags:
    car_data = scrape_car_data(car_url)
    df.loc[len(df)]=car_data

C:\Users\odeya.h\AppData\Local\Temp\ipykernel_7368\1627630181.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)]=car_data
C:\Users\odeya.h\AppData\Local\Temp\ipykernel_7368\1627630181.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)]=car_data
C:\Users\odeya.h\AppData\Local\Temp\ipykernel_7368\1627630181.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty 

In [19]:
df['Repub_date'] = pd.to_datetime(df['Repub_date'], format='%d/%m/%Y')
df['Cre_date'] = pd.to_datetime(df['Cre_date'], format='%d/%m/%Y')

df['Pic_num'] = df['Pic_num'].fillna(0).astype(int)
df['Price'] = df['Price'].astype(float)
#df['Km'] = df['Km'].astype(int)
#df['Test'] = df['Test'].astype(int)

df['Gear'] = df['Gear'].astype('category')
df['Engine_type'] = df['Engine_type'].astype('category')
df['Prev_ownership'] = df['Prev_ownership'].astype('category')
df['Curr_ownership'] = df['Curr_ownership'].astype('category')

In [20]:
# Check the data types of the DataFrame

df.dtypes 

manufactor                 object
Year                        int64
model                      object
Hand                        int64
Gear                     category
capacity_Engine             int64
Engine_type              category
Prev_ownership           category
Curr_ownership           category
Area                       object
City                       object
Price                     float64
Pic_num                    object
Cre_date           datetime64[ns]
Repub_date         datetime64[ns]
Description                object
Color                      object
Km                         object
Test                       object
Supply_score               object
english-model              object
dtype: object

________________________________________

## Supply_score

In [13]:
df['model'].unique()

array(['מאליבו', 'קורסיקה', 'אקווינוקס', 'ספארק', 'קאמרו', 'טראקס',
       'קרוז', 'אפיקה', 'קוואליר', 'סוניק', 'אפלנדר', 'אוואו',
       'קרוז החדשה', 'אימפלה', 'קאמארו', 'אבאו', 'אורלנדו', 'סלבריטי',
       'קורבט', 'אופטרה', 'קורבט Z06', 'אלרו'], dtype=object)

In [14]:
# Dictionary to map Hebrew model names to English model names
hebrew_to_english = {
    'קרוז': 'Cruz',
    'אורלנדו': 'Orlando',
    'מליבו': 'Malibu',
    'ספארק': 'Spark',
    'אקווינוקס': 'Equinox',
    'סוניק': 'Sonic',
    'קורבט': 'Corvette',
    'אבאו': 'Avo',
    'אימפלה': 'Impala',
    'אופטרה': 'Optera',
    'קורבט Z06': 'Corvette Z06',
    'אוואו': 'Aww',
    'קורסיקה': 'Corsica',
    'קאמרו': 'Camaro',
    'טראקס': 'Trax',
    'אפיקה': 'Epica',
    'קוואליר': 'Cavalier',
    'אפלנדר': 'Uplander',
    'קרוז החדשה': 'New Cruz',
    'קאמארו': 'Camaro',
    'סלבריטי': 'Celebrity',
    'אלרו': 'Alero'
}


In [15]:
# Function to convert Hebrew model names to English
def convert_model_name(model_name):
    return hebrew_to_english.get(model_name, model_name)  # Return the English name if found, otherwise return the original name


In [16]:
# Apply the conversion function and create a new column
df['english-model'] = df['model'].apply(convert_model_name)

In [18]:
# URL to the API
api_url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=5e87a7a1-2f6f-41c1-8aec-7216d52a6cf6'

# Fetch data from the API
response = requests.get(api_url)
data = response.json()

# Convert the data to a DataFrame
api_df = pd.DataFrame(data['result']['records'])

KeyError: 'result'

In [ ]:
# Merge the DataFrame with the API data on the relevant columns
merged_df = pd.merge(df, api_df, left_on=['manufactor', 'english-model', 'Year'],
                     right_on=['tozar', 'kinuy_mishari', 'shnat_yitzur'], how='left')

# Calculate the supply index by counting the number of matches
supply_index = merged_df.groupby(['manufactor', 'english-model', 'Year']).size().reset_index(name='supply_index')

# Merge the supply index back into the original DataFrame
df = pd.merge(df, supply_index, on=['manufactor', 'english-model', 'Year'], how='left')

In [ ]:
# Drop the temporary 'english-model' column
df.drop('english-model', axis=1, inplace=True)

In [ ]:
df